## Checking data of AuxTel sequencer change

Craig Lage - 06Oct25

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.time import Time, TimeDelta
from lsst.summit.utils.utils import dayObsIntToString
from lsst.summit.utils.efdUtils import calcNextDay
from lsst.daf.butler import Butler

In [ ]:
butler = Butler('/repo/embargo', collections=["LATISS/raw/all", "LATISS/calib"])
instrument='LATISS'

In [ ]:
startDay = 20250701
endDay = 20250801

dayObs = startDay
expIds = []
while dayObs <= endDay:
    exposureList = []
    for record in butler.registry.queryDimensionRecords("exposure", 
                where=f"exposure.day_obs={dayObs} and instrument='LSSTCam'"):
        exposureList.append([record.id, record])
    exposureList.sort(key=lambda x: x[0])
    #print(len(exposureList))
    for [id,record] in exposureList:
        if record.observation_type not in ['acq', 'engtest', 'cwfs']:
            continue
        try:
            rawExp = butler.get('raw', detector=0, exposure=record.id, instrument=instrument)
            md = rawExp.getMetadata()
            seqName = md['SEQNAME']
            print(f"{record.id} used sequencer {seqName}.")
            break
        except:
            #print(f"{record.id} failed!")
            continue
    dayObs = calcNextDay(dayObs)
    
